In [1]:
!pip uninstall -y torch torchvision torchaudio -qq
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q nnunetv2==2.4.2 monai nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 134.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q nibabel matplotlib pandas

In [ ]:
# 重新安装依赖，但锁定 numpy 版本
# ---- Step ① 已手动点过 Runtime ▸ Restart runtime() ----

# Step ②: 安装统一依赖
!pip install -q --upgrade "numpy>=1.26,<2" nibabel pandas matplotlib scikit-image

# Step ③: 执行前 5 病例推理 + 评估 + 可视化
# （把之前给你的整段脚本粘在这行下面即可）

import json, shutil, subprocess, textwrap, re, os, nibabel as nib
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Image, Markdown
from google.colab import drive

# ---------- 0. 常量 ----------
N_CASES      = 5                    # 取前 N 个病例
MODALITY     = "t1c"                # 叠加底图，可改 t1n/t2f/t2w
DRIVE_ROOT   = Path("/content/drive/MyDrive/data")
TRAIN_DIR    = DRIVE_ROOT / "training_data"
PRED_DIR     = DRIVE_ROOT / "output"          # 预测和 PNG 保存处
PRED_DIR.mkdir(exist_ok=True)

# 挂载 Google Drive
drive.mount("/content/drive", force_remount=False)

# ---------- 1. 选前 N 个病例 ----------
cases = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])[:N_CASES]
assert cases, "❌ training_data 为空！"
print("将处理病例:", cases)

# ---------- 2. 准备 nnUNet 测试集目录 ----------
plans = json.load((DRIVE_ROOT/"plans.json").open())
DATASET  = plans["dataset_name"]
RAW_IMG  = Path("/content/nnUNet_raw") / DATASET / "imagesTs"
RAW_IMG.mkdir(parents=True, exist_ok=True)

MOD_MAP = {"t1c":"0000","t1n":"0001","t2f":"0002","t2w":"0003"}
for case in cases:
    case_dir = TRAIN_DIR / case
    for mod, idx in MOD_MAP.items():
        src = case_dir / f"{case}-{mod}.nii.gz"
        dst = RAW_IMG  / f"{case}_{idx}.nii.gz"
        dst.unlink(missing_ok=True)
        dst.symlink_to(src.resolve())
print("✅ 已软链影像 →", RAW_IMG)

# ---------- 3. 运行推理 ----------
device = "cuda" if Path("/usr/bin/nvidia-smi").exists() else "cpu"
cmd = textwrap.dedent(f"""
    nnUNetv2_predict \
      -i {RAW_IMG} \
      -o /content/pred \
      -d {DATASET} \
      -c 3d_fullres \
      -f 0 \
      -device {device} \
      --disable_tta
""")
print("🚀 推理命令:\n", cmd)
subprocess.run(cmd, shell=True, check=True)

# 把预测复制到 Drive/output
for p in Path("/content/pred").glob("*.nii*"):
    shutil.copy(p, PRED_DIR / p.name)
print("✅ 推理结果已保存 →", PRED_DIR)

# ---------- 4. 评估 & 可视化 ----------
metrics, pngs = [], []
for case in cases:
    pred_fp = PRED_DIR / f"{case}.nii.gz"
    gt_fp   = TRAIN_DIR / case / f"{case}-seg.nii.gz"
    img_fp  = TRAIN_DIR / case / f"{case}-{MODALITY}.nii.gz"
    assert pred_fp.exists() and gt_fp.exists() and img_fp.exists(), f"缺文件: {case}"

    pred = nib.load(pred_fp).get_fdata() > 0
    gt   = nib.load(gt_fp).get_fdata() > 0
    img  = nib.load(img_fp).get_fdata()

    tp = np.logical_and(pred, gt).sum()
    fp = np.logical_and(pred, ~gt).sum()
    fn = np.logical_and(~pred, gt).sum()
    dice = 2*tp / (2*tp + fp + fn + 1e-8)
    iou  = tp / (tp + fp + fn + 1e-8)
    metrics.append({"case": case, "dice": dice, "iou": iou})

    # 中央轴向切片
    z = img.shape[2]//2
    norm = lambda x: (x - x.min())/(x.ptp()+1e-8)
    img_sl, pred_sl, gt_sl = img[...,z], pred[...,z], gt[...,z]

    fig, ax = plt.subplots(1,3, figsize=(12,4))
    ax[0].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[0].imshow(pred_sl.T, cmap='Reds', alpha=.35, origin='lower')
    ax[0].set_title("Prediction"); ax[0].axis('off')

    ax[1].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[1].imshow(gt_sl.T, cmap='Reds', alpha=.35, origin='lower')
    ax[1].set_title("Ground-Truth"); ax[1].axis('off')

    overlay = np.zeros((*pred_sl.T.shape,4))
    overlay[pred_sl.T] = [1,0,0,.35]
    overlay[np.logical_and(gt_sl.T, ~pred_sl.T)] = [0,0,1,.35]
    ax[2].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[2].imshow(overlay, origin='lower')
    ax[2].set_title("Overlap"); ax[2].axis('off')

    fig.suptitle(f"{case}  Dice={dice:.3f}  IoU={iou:.3f}")
    png_path = PRED_DIR / f"{case}_compare.png"
    fig.savefig(png_path, dpi=150, bbox_inches='tight'); plt.close(fig)
    pngs.append(png_path)
    print("📷 保存 →", png_path)

# ---------- 5. 保存 & 展示指标 ----------
metrics_path = PRED_DIR / "metrics_summary.json"
metrics_path.write_text(json.dumps(metrics, indent=2))
print("✅ 指标写入", metrics_path)

df = pd.DataFrame(metrics).sort_values("dice", ascending=False)
display(Markdown("## Segmentation Metrics")); display(df)

display(Markdown("### Overlay comparisons"))
for p in pngs:
    display(Image(filename=str(p)))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
将处理病例: ['BraTS-GLI-02405-100', 'BraTS-GLI-02405-101', 'BraTS-GLI-02406-100', 'BraTS-GLI-02407-100', 'BraTS-GLI-02408-100']
✅ 已软链影像 → /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs
🚀 推理命令:
 
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset241_BraTS_2024_Real/image

CalledProcessError: Command '
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs       -o /content/pred       -d Dataset241_BraTS_2024_Real       -c 3d_fullres       -f 0       -device cpu       --disable_tta
' returned non-zero exit status 1.

In [ ]:
import subprocess, textwrap, pathlib, os, json, re, shutil
from IPython.display import Markdown

cmd = textwrap.dedent("""
    nnUNetv2_predict \
      -i /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs \
      -o /content/pred \
      -d Dataset241_BraTS_2024_Real \
      -c 3d_fullres \
      -f 0 \
      -device cpu \
      --disable_tta
""")

res = subprocess.run(cmd, shell=True, text=True,
                     capture_output=True)
print("---- STDOUT ----\n", res.stdout)
print("---- STDERR ----\n", res.stderr)

---- STDOUT ----
 nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep 

In [ ]:
 # ======================== nnUNetv2 一键推理 =========================
# 放在 Colab 单元格直接运行。Google Drive 中 data/ 目录需包含：
#  ├─ plans.json
#  ├─ dataset.json               ← 可以是 v1 或 v2 格式，脚本会自动修
#  ├─ checkpoint_final.pth       ← 训练好的权重
#  ├─ BraTS-****-t1c.nii.gz …    ← 四通道影像文件
#  └─ output/                    ← 推理结果将复制到这里（若无会创建）
# ===================================================================

import json, shutil, subprocess, textwrap, re, os
from pathlib import Path
from google.colab import drive

# ---------- 0. 挂载 Google Drive ----------
drive.mount("/content/drive", force_remount=False)
DRIVE = Path("/content/drive/MyDrive/data").resolve()

# ---------- 1. 解析 plans.json ----------
plans_path = DRIVE / "plans.json"
assert plans_path.exists(), "❌ plans.json 不存在于 Drive/data！"
DATASET = json.load(plans_path.open())["dataset_name"]
print("📌 数据集名:", DATASET)

# ---------- 2. 目录准备 ----------
RAW_DIR   = Path("/content/nnUNet_raw") / DATASET
RAW_IMG   = RAW_DIR / "imagesTs"
RES_DIR   = Path("/content/nnUNet_results") / DATASET / "nnUNetTrainer__nnUNetPlans__3d_fullres"
RAW_IMG.mkdir(parents=True, exist_ok=True)
RES_DIR.mkdir(parents=True, exist_ok=True)

# ---------- 3. 软链影像文件 ----------
MOD_MAP = {"t1c": "0000", "t1n": "0001", "t2f": "0002", "t2w": "0003"}
for img in DRIVE.glob("*.nii*"):
    m = re.match(r"(.+)-([a-z0-9]{3})(?:\.nii(\.gz)?)$", img.name)
    if m and m.group(2) in MOD_MAP:
        link = RAW_IMG / f"{m.group(1)}_{MOD_MAP[m.group(2)]}.nii.gz"
        link.unlink(missing_ok=True)
        link.symlink_to(img.resolve())
print("✅ 已软链影像 →", RAW_IMG)

# ---------- 4. 同步 & 修正 dataset.json ----------
LABELS_FIXED = {          # !!! 根据权重输出通道数修改 !!!
    "background": 0,
    "whole_tumor": 1,
    "tumor_core": 2,
    "enhancing": 3
}

ds_src = DRIVE / "dataset.json"
assert ds_src.exists(), "❌ dataset.json 不存在于 Drive/data！"
ds_dst = RAW_DIR / "dataset.json"
shutil.copy(ds_src, ds_dst)

# 自动把 v1 风格 (数字→名字) 反转为 v2 风格 (名字→数字)
with ds_dst.open() as f:
    ds = json.load(f)
if "background" not in ds["labels"]:
    ds["labels"] = {v: int(k) for k, v in ds["labels"].items()}

# 覆盖为固定标签（确保通道数匹配）
ds["labels"] = LABELS_FIXED
with ds_dst.open("w") as f:
    json.dump(ds, f, indent=2)
shutil.copy(ds_dst, RES_DIR / "dataset.json")
print("✅ dataset.json 已修正并同步")

# ---------- 5. 复制 plans.json & 权重 ----------
shutil.copy(plans_path, RES_DIR / "plans.json")
(RES_DIR / "fold_0").mkdir(exist_ok=True)
shutil.copy(DRIVE / "checkpoint_final.pth",
            RES_DIR / "fold_0/checkpoint_final.pth")

# ---------- 6. 设备选择 ----------
device = "cuda" if Path("/usr/bin/nvidia-smi").exists() else "cpu"
print("📌 推理设备:", device.upper())

# ---------- 7. 执行推理 ----------
OUT_DIR = Path("/content/pred")
cmd = textwrap.dedent(f"""
    nnUNetv2_predict \
      -i {RAW_IMG} \
      -o {OUT_DIR} \
      -d {DATASET} \
      -c 3d_fullres \
      -f 0 \
      -device {device} \
      --disable_tta
""")
print("🚀 运行命令:\n", cmd)
subprocess.run(cmd, shell=True, check=True)
print("✅ 推理完成！结果在", OUT_DIR)

# ---------- 8. 结果复制回 Drive ----------
( DRIVE / "output").mkdir(exist_ok=True)
for p in OUT_DIR.glob("*.nii*"):
    shutil.copy(p, DRIVE / "output" / p.name)
print(f"✅ 已复制 {len(list(OUT_DIR.glob('*.nii*')))} 个文件 → {DRIVE/'output'}")

Mounted at /content/drive
📌 数据集名: Dataset241_BraTS_2024_Real
✅ 已软链影像 → /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs
✅ dataset.json 已修正并同步
📌 推理设备: CPU
🚀 运行命令:
 
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs       -o /content/pred       -d Dataset241_BraTS_2024_Real       -c 3d_fullres       -f 0       -device cpu       --disable_tta



CalledProcessError: Command '
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset241_BraTS_2024_Real/imagesTs       -o /content/pred       -d Dataset241_BraTS_2024_Real       -c 3d_fullres       -f 0       -device cpu       --disable_tta
' returned non-zero exit status 1.

In [1]:
!pip install -q --upgrade --force-reinstall numpy pandas==2.2.2 matplotlib nibabel
!pip uninstall -y torch torchvision torchaudio -qq
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q nnunetv2==2.4.2 monai nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 k

In [1]:

#  ├─ /content/drive/MyDrive/data/
#       ├─ plans.json
#       ├─ dataset.json
#       ├─ checkpoint_final.pth
#       ├─ training_data/
#           ├─ CaseA/CaseA-t1c.nii.gz … CaseA-seg.nii.gz
#           ├─ CaseB/…
#       ├─ output/


# 安装依赖
!pip install -q nibabel matplotlib pandas

import os
import json
import shutil
import subprocess
import textwrap
import re
from pathlib import Path

import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image, Markdown
from google.colab import drive
drive.mount("/content/drive", force_remount=False)
# ---------- 0. 常量 & 挂载 Drive ----------
N_CASES    = 5
MODALITY   = "t1c"   # 如果要用 t1n、t2f、t2w，请改这里
DRIVE_ROOT = Path("/content/drive/MyDrive/data")
TRAIN_DIR  = DRIVE_ROOT / "training_data"
OUTPUT_DIR = DRIVE_ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)



# ---------- 1. 解析 plans.json & 设置 nnUNet 环境变量 ----------
plans = json.load((DRIVE_ROOT/"plans.json").open())
DATASET = plans["dataset_name"]

# 确保 nnUNet v2 能找到目录
for d in ("/content/nnUNet_raw", "/content/nnUNet_preprocessed", "/content/nnUNet_results"):
    Path(d).mkdir(parents=True, exist_ok=True)
os.environ.update({
    "nnUNet_raw": "/content/nnUNet_raw",
    "nnUNet_preprocessed": "/content/nnUNet_preprocessed",
    "nnUNet_results": "/content/nnUNet_results",
    "RESULTS_FOLDER": "/content/nnUNet_results"
})

# ---------- 2. 选前 N_CASES 病例 & 软链影像到 nnUNet_raw ----------
cases = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])[:N_CASES]
assert cases, "training_data 目录中没有病例文件夹！"
print("处理病例:", cases)

IMG_TS = Path("/content/nnUNet_raw")/DATASET/"imagesTs"
IMG_TS.mkdir(parents=True, exist_ok=True)
MOD_MAP = {"t1c":"0000","t1n":"0001","t2f":"0002","t2w":"0003"}

for case in cases:
    src_dir = TRAIN_DIR/case
    for mod, idx in MOD_MAP.items():
        src = src_dir/f"{case}-{mod}.nii.gz"
        dst = IMG_TS/f"{case}_{idx}.nii.gz"
        dst.unlink(missing_ok=True)
        dst.symlink_to(src.resolve())

print("✅ 已软链前五病例影像至", IMG_TS)

# ---------- 3. 执行 nnUNetv2_predict ----------
device = "cuda" if Path("/usr/bin/nvidia-smi").exists() else "cpu"
cmd = textwrap.dedent(f"""
    nnUNetv2_predict \
      -i {IMG_TS} \
      -o /content/pred \
      -d {DATASET} \
      -c 3d_fullres \
      -f 0 \
      -device {device} \
      --disable_tta
""")
print("🚀 运行命令：", cmd)
subprocess.run(cmd, shell=True, check=True)
print("✅ 推理完成 → /content/pred")

# 复制预测结果到 Drive/output
for p in Path("/content/pred").glob("*.nii*"):
    shutil.copy(p, OUTPUT_DIR/p.name)
print("✅ 预测结果已复制到", OUTPUT_DIR)

# ---------- 4. 评估 & 可视化 ----------
metrics = []
png_paths = []

for case in cases:
    pred_fp = OUTPUT_DIR/f"{case}.nii.gz"
    gt_fp   = TRAIN_DIR/case/f"{case}-seg.nii.gz"
    img_fp  = TRAIN_DIR/case/f"{case}-{MODALITY}.nii.gz"
    assert pred_fp.exists() and gt_fp.exists() and img_fp.exists(), f"{case} 文件不完整"

    pred = nib.load(pred_fp).get_fdata() > 0
    gt   = nib.load(gt_fp).get_fdata() > 0
    img  = nib.load(img_fp).get_fdata()

    tp = np.logical_and(pred, gt).sum()
    fp = np.logical_and(pred, ~gt).sum()
    fn = np.logical_and(~pred, gt).sum()
    dice = 2*tp / (2*tp + fp + fn + 1e-8)
    iou  = tp / (tp + fp + fn + 1e-8)
    metrics.append({"case": case, "dice": float(dice), "iou": float(iou)})

    # 中央轴向切片可视化
    z = img.shape[2]//2
    img_sl, pred_sl, gt_sl = img[:,:,z], pred[:,:,z], gt[:,:,z]
    norm = lambda x: (x - x.min())/(x.ptp()+1e-8)

    fig, ax = plt.subplots(1,3, figsize=(12,4))
    ax[0].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[0].imshow(pred_sl.T, cmap='Reds', alpha=.35, origin='lower')
    ax[0].set_title("Prediction"); ax[0].axis('off')

    ax[1].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[1].imshow(gt_sl.T, cmap='Reds', alpha=.35, origin='lower')
    ax[1].set_title("Ground-Truth"); ax[1].axis('off')

    overlay = np.zeros((*pred_sl.T.shape,4))
    overlay[pred_sl.T] = [1,0,0,.35]
    overlay[np.logical_and(gt_sl.T, ~pred_sl.T)] = [0,0,1,.35]
    ax[2].imshow(norm(img_sl).T, cmap='gray', origin='lower')
    ax[2].imshow(overlay, origin='lower')
    ax[2].set_title("Overlap"); ax[2].axis('off')

    fig.suptitle(f"{case}  Dice={dice:.3f}  IoU={iou:.3f}")
    png_fp = OUTPUT_DIR/f"{case}_compare.png"
    fig.savefig(png_fp, dpi=150, bbox_inches='tight')
    plt.close(fig)
    png_paths.append(png_fp)
    print("📷 保存对比图 →", png_fp)

# ---------- 5. 保存指标 & 展示 ----------
metrics_fp = OUTPUT_DIR/"metrics_summary.json"
metrics_fp.write_text(json.dumps(metrics, indent=2))
print("✅ 指标保存 →", metrics_fp)

# 在 Notebook 中展示
df = pd.DataFrame(metrics).sort_values("dice", ascending=False)
display(Markdown("## Segmentation Metrics"))
display(df)

display(Markdown("### Overlay Comparisons"))
for p in png_paths:
    display(Image(str(p)))

Mounted at /content/drive
处理病例: ['BraTS-GLI-00000-000', 'BraTS-GLI-00002-000', 'BraTS-GLI-00003-000', 'BraTS-GLI-00005-000', 'BraTS-GLI-00006-000']
✅ 已软链前五病例影像至 /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs
🚀 运行命令： 
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs       -o /content/pred       -d Dataset232_BraTS_2023_rGANs       -c 3d_fullres       -f 0       -device cpu       --disable_tta



CalledProcessError: Command '
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs       -o /content/pred       -d Dataset232_BraTS_2023_rGANs       -c 3d_fullres       -f 0       -device cpu       --disable_tta
' returned non-zero exit status 1.

In [3]:
# 2. 导入 & 挂载 Drive
import os, shutil, subprocess, textwrap
from pathlib import Path
from google.colab import drive

drive.mount("/content/drive", force_remount=False)

# 3. 环境变量配置
os.environ.update({
    "nnUNet_raw": "/content/nnUNet_raw",
    "nnUNet_preprocessed": "/content/nnUNet_preprocessed",
    "nnUNet_results": "/content/nnUNet_results",
    "RESULTS_FOLDER": "/content/nnUNet_results"
})

# 4. 路径设定
DRIVE_ROOT = Path("/content/drive/MyDrive/data")
DATASET    = "Dataset232_BraTS_2023_rGANs"   # 请和你的 plans.json 中 dataset_name 保持一致

# 5. 准备 nnUNet_raw：软链前处理原始影像
RAW_TS = Path(os.environ["nnUNet_raw"]) / DATASET / "imagesTs"
RAW_TS.mkdir(parents=True, exist_ok=True)

MOD_MAP = {"t1c":"0000","t1n":"0001","t2f":"0002","t2w":"0003"}
for case_dir in (DRIVE_ROOT/"training_data").iterdir():
    case = case_dir.name
    for mod, idx in MOD_MAP.items():
        src = case_dir / f"{case}-{mod}.nii.gz"
        dst = RAW_TS   / f"{case}_{idx}.nii.gz"
        dst.unlink(missing_ok=True)
        dst.symlink_to(src.resolve())
print("✅ 已软链原始影像 →", RAW_TS)

# 6. 准备 nnUNet_preprocessed：拷贝 dataset.json、plans.json 及 fold_* 目录
PREPROC_DST = Path(os.environ["nnUNet_preprocessed"]) / DATASET
PREPROC_DST.mkdir(parents=True, exist_ok=True)
for fn in ["dataset.json","plans.json"]:
    shutil.copy(DRIVE_ROOT/fn, PREPROC_DST/fn)
for fold in DRIVE_ROOT.glob("fold_*"):
    if fold.is_dir():
        shutil.copytree(fold, PREPROC_DST/fold.name, dirs_exist_ok=True)
print("✅ 已拷贝预处理数据 →", PREPROC_DST)

# 7. 准备 nnUNet_results：构建模型输出目录并放入 dataset.json、plans.json、checkpoint_final.pth
RES_DST   = Path(os.environ["nnUNet_results"]) / DATASET
# 以下两个标识符请根据你训练时实际用的 trainer 和 plans 变化，
# 这里示例中脚本默认要找的是：nnUNetTrainer__nnUNetPlans__3d_fullres
TR_DIR    = RES_DST / "nnUNetTrainer__nnUNetPlans__3d_fullres"
FOLD0_DIR = TR_DIR  / "fold_0"
FOLD0_DIR.mkdir(parents=True, exist_ok=True)

shutil.copy(DRIVE_ROOT/"dataset.json", TR_DIR/"dataset.json")
shutil.copy(DRIVE_ROOT/"plans.json",   TR_DIR/"plans.json")
shutil.copy(DRIVE_ROOT/"checkpoint_final.pth", FOLD0_DIR/"checkpoint_final.pth")
print("✅ 已拷贝模型权重 & 配置 →", TR_DIR)

# 8. 运行推理
device = "cuda" if Path("/usr/bin/nvidia-smi").exists() else "cpu"
cmd = textwrap.dedent(f"""
    nnUNetv2_predict \
      -i {RAW_TS} \
      -o /content/pred \
      -d {DATASET} \
      -c 3d_fullres \
      -f 0 \
      -device {device} \
      --disable_tta
""")
print("🚀 Running:", cmd)
subprocess.run(cmd, shell=True, check=True)
print("✅ 推理完成 → /content/pred")

# 9. 复制预测结果回 Drive/output
OUT_DIR = DRIVE_ROOT / "output"
OUT_DIR.mkdir(exist_ok=True)
for p in Path("/content/pred").glob("*.nii*"):
    shutil.copy(p, OUT_DIR/p.name)
print("✅ 预测结果已复制到", OUT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 已软链原始影像 → /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs
✅ 已拷贝预处理数据 → /content/nnUNet_preprocessed/Dataset232_BraTS_2023_rGANs
✅ 已拷贝模型权重 & 配置 → /content/nnUNet_results/Dataset232_BraTS_2023_rGANs/nnUNetTrainer__nnUNetPlans__3d_fullres
🚀 Running: 
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs       -o /content/pred       -d Dataset232_BraTS_2023_rGANs       -c 3d_fullres       -f 0       -device cpu       --disable_tta



CalledProcessError: Command '
nnUNetv2_predict       -i /content/nnUNet_raw/Dataset232_BraTS_2023_rGANs/imagesTs       -o /content/pred       -d Dataset232_BraTS_2023_rGANs       -c 3d_fullres       -f 0       -device cpu       --disable_tta
' returned non-zero exit status 1.

In [4]:
completed = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print(completed.stdout)
print(completed.stderr)   # 真正的 traceback 在这里


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

perform_everything_on_device=True is only supported for cuda devices! Setting this to False

/usr/local/lib/python3.11/dist-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flip